In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import pylab as plt
import random
from scipy import stats
from sklearn.neighbors import KDTree
import time
from sklearn.metrics import mean_squared_error
from astropy.cosmology import FlatLambdaCDM
from os import listdir
import desispec
import desispec.io
import speclite.filters
import scipy
from desitarget.targetmask import desi_mask, bgs_mask, mws_mask
from desitarget.cmx.cmx_targetmask import cmx_mask

In [ ]:
zall_path="/project/projectdirs/desi/spectro/redux/everest/zcatalog/ztile-main-bright-cumulative.fits"
data=Table.read(zall_path,hdu=1)

In [3]:
#len(np.where(data["SPECTYPE"]=='QSO')[0])
# target_pos=np.where((data["BGS_TARGET"] & bgs_mask.mask("BGS_BRIGHT")))[0]
# target_ids=data["TARGETID"][target_pos]
# fiber_ids=data["FIBER"][target_pos]
# petal_locs=data["PETAL_LOC"][target_pos]
# tile_ids=data["TILEID"][target_pos]

# i=0
# coadd_path="/project/projectdirs/desi/spectro/redux/everest/tiles/cumulative/"+str(tile_ids[0])
# a=listdir(coadd_path)[0]
# coadd_path="/project/projectdirs/desi/spectro/redux/everest/tiles/cumulative/"+str(tile_ids[0])+"/"+a
# coadd_path="/project/projectdirs/desi/spectro/redux/everest/tiles/cumulative/"+str(tile_ids[0])+"/"+a\
#             +"/coadd-"+str(petal_locs[0])+"-"+str(tile_ids[0])+"-thru"+str(a)+".fits"
# data2=Table.read(coadd_path,hdu=1)

N=len(data['TARGETID'])
positions=[]
for i in range(N):
    k=0
    bgs_test=0
    if (data["BGS_TARGET"][i] & bgs_mask.mask("BGS_BRIGHT"))>0:
        bgs_test+=1
    if data["SPECTYPE"][i]!="GALAXY":
        k+=1
    if data["DELTACHI2"][i]<25:
        k+=1
    if data["Z"][i]<0:
        k+=1
    if data["ZWARN"][i]!=0:
        k+=1
    if k==0 and bgs_test==1:
        positions.append(i)



In [4]:
n=50*10**3
target_pos=np.zeros(n,int)
target_pos[:]=positions[0:n]
target_ids=data["TARGETID"][target_pos]
fiber_ids=data["FIBER"][target_pos]
petal_locs=data["PETAL_LOC"][target_pos]
tile_ids=data["TILEID"][target_pos]
zs=data["Z"][target_pos]

np.savez_compressed("/global/homes/a/ashodkh/results/zs.txt",zs)

In [ ]:
nw=7781
spectra=np.zeros([n,nw])
redshifted_waves=np.zeros([n,nw])
tic=time.time()
for j in range(n):
    coadd_path="/project/projectdirs/desi/spectro/redux/everest/tiles/cumulative/"+str(tile_ids[j])
    a=listdir(coadd_path)[0]
    coadd_path="/project/projectdirs/desi/spectro/redux/everest/tiles/cumulative/"+str(tile_ids[j])+"/"+a
    coadd_path="/project/projectdirs/desi/spectro/redux/everest/tiles/cumulative/"+str(tile_ids[j])+"/"+a\
                +"/coadd-"+str(petal_locs[j])+"-"+str(tile_ids[j])+"-thru"+str(a)+".fits"


    test=fits.open(coadd_path)
    wave_b=test["B_WAVELENGTH"].data
    flux_b=test["B_FLUX"].data[fiber_ids[j]%500,:]
    inv_var_b=test["B_IVAR"].data[fiber_ids[j]%500,:]
    wave_r=test["R_WAVELENGTH"].data
    flux_r=test["R_FLUX"].data[fiber_ids[j]%500,:]
    inv_var_r=test["R_IVAR"].data[fiber_ids[j]%500,:]
    wave_z=test["Z_WAVELENGTH"].data
    flux_z=test["Z_FLUX"].data[fiber_ids[j]%500,:]
    inv_var_z=test["Z_IVAR"].data[fiber_ids[j]%500,:]

    first=np.where(abs(wave_b-wave_r[0])<10**-4)[0][0]
    second=np.where(abs(wave_r-wave_z[0])<10**-4)[0][0]

    nw=first+second+len(wave_z)

    spectrum=np.zeros(nw)

    i2=np.arange(nw)-first
    i3=np.arange(nw)-(second+first)

    wavelength=np.zeros(nw)
    
    wavelength[:first]=wave_b[:first]
    wavelength[first:len(wave_b)]=(wave_b[first:len(wave_b)]+wave_r[i2[first:len(wave_b)]])/2
    wavelength[len(wave_b):second+first]=wave_r[i2[len(wave_b):second+first]]
    wavelength[second+first:first+len(wave_r)]=(wave_r[i2[second+first:first+len(wave_r)]]+wave_z[i3[second+first:first+len(wave_r)]])/2
    wavelength[first+len(wave_r):]=wave_z[i3[first+len(wave_r):]]
        
    spectrum[:first]=flux_b[:first]  
    spectrum[first:len(wave_b)]=(flux_b[first:len(wave_b)]*inv_var_b[first:len(wave_b)]+flux_r[i2[first:len(wave_b)]]*inv_var_r[i2[first:len(wave_b)]])\
                                /(inv_var_b[first:len(wave_b)]+inv_var_r[i2[first:len(wave_b)]])
    spectrum[len(wave_b):second+first]=flux_r[i2[len(wave_b):second+first]]
    spectrum[second+first:first+len(wave_r)]=(flux_r[i2[second+first:first+len(wave_r)]]*inv_var_r[i2[second+first:first+len(wave_r)]]+flux_z[i3[second+first:first+len(wave_r)]]*inv_var_z[i3[second+first:first+len(wave_r)]])\
                    /(inv_var_r[i2[second+first:first+len(wave_r)]]+inv_var_z[i3[second+first:first+len(wave_r)]])
    spectrum[first+len(wave_r):]=flux_z[i3[first+len(wave_r):]]

#     for i in range(nw):
#         if i<first:
#             wavelength[i]=wave_b[i]
#             spectrum[i]=flux_b[i]

#         if i>=first and i<len(wave_b):
#             wavelength[i]=(wave_b[i]+wave_r[i-first])/2
#             spectrum[i]=(flux_b[i]*inv_var_b[i]+flux_r[i-first]*inv_var_r[i-first])/(inv_var_b[i]+inv_var_r[i-first])

#         if i>=len(wave_b) and i<second+first:
#             wavelength[i]=wave_r[i-first]
#             spectrum[i]=flux_r[i-first]

#         if i>=second+first and i<first+len(wave_r):
#             wavelength[i]=(wave_r[i-first]+wave_z[i-(second+first)])/2
#             spectrum[i]=(flux_r[i-first]*inv_var_r[i-first]+flux_z[i-(second+first)]*inv_var_z[i-(second+first)])/(inv_var_r[i-first]+inv_var_z[i-(second+first)])

#         if i>=first+len(wave_r):
#             wavelength[i]=wave_z[i-(first+second)]
#             spectrum[i]=flux_z[i-(first+second)]
    
    spectra[j,:]=spectrum[:]
    redshifted_waves[j,:]=wavelength[:]/(1+zs[j])
print(time.time()-tic)

plt.figure(1)
plt.plot(wavelength,spectra[0,:])
plt.figure(2)
plt.plot(wavelength,spectra[-1,:])

In [7]:
# with open("/global/homes/a/ashodkh/results/spectra.txt",'w') as f:    
#     for i in range(n):
#         for j in range(nw-1):
#             f.write(str(np.round(spectra[i,j],decimals=4))+",")
#         f.write(str(np.round(spectra[i,-1],decimals=4))+"\n")
        
# with open("/global/homes/a/ashodkh/results/redshifted_waves.txt",'w') as f:    
#     for i in range(n):
#         for j in range(nw-1):
#             f.write(str(np.round(redshifted_waves[i,j],decimals=4))+",")
#         f.write(str(np.round(redshifted_waves[i,-1],decimals=4))+"\n")   
np.savez_compressed("/global/homes/a/ashodkh/results/spectra.txt",spectra)
np.savez_compressed("/global/homes/a/ashodkh/results/redshifted_waves.txt",redshifted_waves)